# [실습] Langchain으로 시장조사 문서 기반 챗봇 만들기 - PDF

## 실습 목표
---
[실습] LangChain을 활용해서 입력된 문서를 요약해서 Context로 활용하는 챗봇을 개발합니다.

## 실습 목차
---

1. **시장조사 문서 벡터화:** RAG 챗봇에서 활용하기 위해 시장조사 파일을 읽어서 벡터화하는 과정을 실습합니다.

2. **RAG 체인 구성:** 이전 실습에서 구성한 미니 RAG 체인을 응용해서 간단한 시장 조사 문서 기반 RAG 체인을 구성합니다.

3. **챗봇 구현 및 사용:** 구성한 RAG 체인을 활용해서 시장조사 문서 기반 챗봇을 구현하고 사용해봅니다.

## 실습 개요
---
RAG 체인을 활용해서 시장조사 문서 기반 챗봇을 구현하고 사용해봅니다.

## 0. 환경 설정
- 필요한 라이브러리를 불러옵니다.

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

- Ollama를 통해 Mistral 7B 모델을 불러옵니다.

In [2]:
!ollama pull mistral:7b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pull

## 1. 시장조사 문서 벡터화
- RAG 챗봇에서 활용하기 위해 시장조사 파일을 읽어서 벡터화하는 과정을 실습합니다.

먼저, mistral:7b 모델을 사용하는 ChatOllama 객체와 OllamaEmbeddings 객체를 생성합니다.

In [3]:
llm = ChatOllama(model="mistral:7b")
embeddings = OllamaEmbeddings(model="mistral:7b")

다음으로, 시장조사 PDF 문서를 불러와서 벡터화 해보겠습니다.
- 한국소비자원의 2022년 키오스크(무인정보단말기) 이용 실태조사 보고서를 활용했습니다
  - https://www.kca.go.kr/smartconsumer/sub.do?menukey=7301&mode=view&no=1003409523&page=2&cate=00000057
- 이 실태조사 보고서는 2022년 키오스크의 사용자 경험, 접근성, 후속 조치에 대해 논의하는 보고서입니다. 
- 이를 활용해서 키오스크를 어떻게 세일즈 할 수 있을지 아이디어를 제공하는 챗봇을 만들어야 하는 상황이라고 가정해 봅시다.

먼저, LangChain의 `PyPDFLoader`를 활용해서 시장조사 보고서의 텍스트를 추출하고, 페이지 별로 `Document`를 생성하여 저장합니다.

In [4]:
doc_path = "docs/키오스크(무인정보단말기) 이용실태 조사.pdf"
loader = PyPDFLoader(doc_path)
docs = loader.load()

생성된 Document의 수를 확인해봅시다.

In [5]:
print(len(docs))

59


다음으로, 각 Document의 길이를 확인해봅시다.

In [6]:
doc_len = [len(doc.page_content) for doc in docs]
print(doc_len)

[86, 6679, 6011, 5915, 1201, 908, 676, 1289, 821, 1154, 1439, 447, 1031, 1178, 514, 1083, 968, 1119, 1006, 1094, 978, 916, 1230, 862, 680, 1251, 1433, 1290, 729, 1170, 1011, 598, 733, 966, 934, 1195, 514, 1210, 777, 635, 651, 771, 837, 397, 953, 877, 548, 1022, 1198, 1183, 1230, 838, 533, 1255, 1231, 1894, 777, 798, 662]


1천자 미만의 문서도 있지만, 6천자가 넘는 문서도 있는 것을 확인할 수 있습니다. 이대로 그냥 사용할 경우, Context가 너무 길어져 오히려 성능이 낮아질 수도 있습니다.

우선은 이대로 RAG 체인을 구성해 봅시다.

## 2. RAG 체인 구성
RAG 체인을 구성하기 위해 `Document`를 `OllamaEmbeddings`를 활용해 벡터로 변환하고, FAISS DB를 활용하여 저장합니다.
- 변환 및 저장 과정은 약 3분 정도 소요됩니다.

In [7]:
vectorstore = FAISS.from_documents(
    docs,
    embedding=embeddings
)

In [8]:
db_retriever = vectorstore.as_retriever()

이전 실습에서 구성한 미니 RAG Chain과 비슷하게 Chain을 구성해 봅시다.
- 지난 실습과 달리 이번 챗봇의 역할은 마케터를 위한 챗봇으로 고정했으므로, 역할을 별도로 인자로 전달할 필요가 없습니다.
- `RunnablePassthrough()`는 Chain의 이전 구성 요소에서 전달된 값을 그대로 전달하는 역할을 수행합니다.

In [9]:
def get_retrieved_text(docs):
    result = "\n".join([doc.page_content for doc in docs])
    return result

def init_chain():
    messages_with_contexts = [
        ("system", "당신은 마케터를 위한 친절한 지원 챗봇입니다. 사용자가 입력하는 정보를 바탕으로 질문에 답하세요."),
        ("human", "정보: {context}.\n{question}."),
    ]

    prompt_with_context = ChatPromptTemplate.from_messages(messages_with_contexts)

    # 체인 구성
    # context에는 질문과 가장 비슷한 문서를 반환하는 db_retriever에 get_retrieved_text를 적용한 chain의 결과값이 전달됩니다.
    qa_chain = (
        {"context": db_retriever | get_retrieved_text, "question": RunnablePassthrough()}
        | prompt_with_context
        | llm
        | StrOutputParser()
    )
    
    return qa_chain

In [10]:
qa_chain = init_chain()

Chain 구성이 완료되었습니다.

## 3. 챗봇 구현 및 사용
- 구성한 RAG 체인을 활용해서 시장조사 문서 기반 챗봇을 구현하고 사용해봅니다.

방금 구현한 RAG Chain을 사용해서 시장조사 문서 기반 챗봇을 구현해볼 것입니다. 

그 전에, 별도로 RAG 기능을 추가하지 않은 LLM과 답변의 퀄리티를 비교해 봅시다.

In [11]:
messages_with_variables = [
    ("system", "당신은 마케터를 위한 친절한 지원 챗봇입니다."),
    ("human", "{question}."),
]
prompt = ChatPromptTemplate.from_messages(messages_with_variables)
parser = StrOutputParser()
chain = prompt | llm | parser

In [12]:
print(chain.invoke("키오스크 관련 설문조사 결과를 알려줘"))

 안녕하세요! 제가 마케팅 채널 중 한 가지로, 키오스크(Kiosk)에 대한 관련성을 갖추고 있는 설문조사 결과를 소개해드리겠습니다.

1. 키오스크의 활용도
   - 52% 의 참여자들은 키오스크에서 상품 정보를 얻거나 서비스를 요청한 경험을 하였습니다.
   - 38% 의 참여자들은 키오스크로 결제를 하였고, 24%는 선호하는 방법으로 매출이 이루어진 경험을 하였습니다.
   - 50% 의 참여자들은 키오스크로 정보 조회와 결제에 대한 익숙함을 느꼈으며, 42%는 편리하다고 평가했습니다.

2. 키오스크의 불만점
   - 53% 의 참여자들은 키오스크의 이용 방법에 대한 설명이 부족하다고 평가했습니다.
   - 40% 의 참여자들은 키오스크의 화면 구조나 이용성이 어려운 경험을 하였습니다.
   - 39% 의 참여자들은 키오스크로 인해 대기시간이 길었거나, 많은 사람이 동시에 이용한 경험을 하였습니다.

3. 향후 요구사항
   - 82% 의 참여자들은 키오스크를 더욱 편리하게 만드는 것을 원합니다.
   - 75% 의 참여자들은 키오스크의 이용 방법에 대한 설명이 명확해져야 함을 요구했습니다.
   - 63% 의 참여자들은 키오스크로 인터넷 접속과 즐길 수 있는 컨텐츠를 제공하고 싶어한다고 평가했습니다.

이러한 설문조사 결과는 키오스크의 개선 및 발전을 위한 기반으로, 향후 마케팅 전략에 도움이 될 수 있습니다.


In [13]:
print(qa_chain.invoke("키오스크 관련 설문조사 결과를 알려줘"))

 키오스크(무인정보단말기) 관련 설문조사 결과는 다음과 같습니다.

1. 전체 응답자 중 키오스크를 이용한 경험이 있는 사람들의 비율은 80%입니다. (233명 중 186명)
2. 피해 경험 유형별로 분석하면 다음과 같습니다.

   * 강제 이용: 매장에 직원이 있는데도 키오스크로만 거래가 가능하다고 안내함 - 87% (163명)
   * 취소 불가: 주문 후 취소가 불가능함 - 23% (53명)
   * 변경 불가: 결제 전 장바구니에 담긴 상품 또는 서비스를 변경하지 못함 - 30% (69명)
   * 주문 실수: 주문 실수를 인지하지 못해 주문한 것과 다른 상품‧서비스를 받음 - 71% (164명)
   * 상품 미제공: 기기 오류 등으로 상품‧서비스를 받지 못했는데도 결제가 됨. - 77% (170명)

3. 키오스크 관련 피해 경험 여부와 유형은 [그림5-2-1]에서 확인할 수 있습니다.

이 데이터는 29시장조사국 시장감시팀의 233명의 응답자로부터 수집한 정보입니다.


일반 체인은 아무런 출처가 없는 답변을 생성한 반면, RAG 기능을 추가한 챗봇은 데이터를 기반으로 상대적으로 정확한 답변을 하는 것을 확인할 수 있습니다. 

이제 챗봇을 한번 사용해 봅시다.

In [ ]:
qa_chain = init_chain()
while True:
    question = input("질문을 입력해주세요 (종료를 원하시면 '종료'를 입력해주세요.): ")
    if question == "종료":
        break
    else:
        result = qa_chain.invoke(question)
        print(result)

질문을 입력해주세요 (종료를 원하시면 '종료'를 입력해주세요.): 키오스크 사용을 어려워하는 나이는 몇 살인가요?
 키오스크를 사용하는 것이 어려운 나이는 65세 이상으로 추정된다. 김효정의 연구에 따르면 노년층(65세 이상) 시니어들은 키오스크 이용 경험이 부정적이었고, 디지털 약자층인 장애인과 고령자의 비율도 지속적으로 증가하고 있어서 키오스크 환경에서는 많은 불편이 제기되고 있다는 것으로 알려져 있습니다.
질문을 입력해주세요 (종료를 원하시면 '종료'를 입력해주세요.): 키오스크 사용 약자를 도와주는 방법은 무엇이 있을까요?
 The text you provided is a study on the use of kiosks (self-service information terminals) and their accessibility for people with disabilities in Korea. Here's a summary:

1. The "Act on the Promotion of the Use of Kiosks" came into effect on January 28, 2023. However, kiosks installed before this date will be exempt from providing accessibility features for disabled individuals for three years. This means that the mandatory requirement to provide accessibility for the disabled through kiosks is delayed until 2026.

2. The global kiosk market is expected to grow by an average of 9.8% annually, reaching approximately $44 trillion by 2027. In Korea, the number of kiosks in the private sector

저희는 이전 챕터에서 구현한 챗봇이 가지고 있는 문제점 중 '문서나 데이터 기반 추론이 불가능하다.'를 완화했습니다.

또한, 지금 구성한 챗봇은 UI가 없고 단순 표준 입출력 만을 사용합니다. 5챕터에서 Streamlit을 활용해 ChatGPT와 비슷한 웹 챗봇 어플리케이션을 제작해 볼 것입니다.